In [ ]:
import pandas as pd

questions_path = "../data/raw/Questions.csv"

questions_df = pd.read_csv(
    questions_path,
    encoding="latin1",
    nrows=10000      # chunk
)

questions_df.shape 





(1264216, 7)

In [4]:
import pandas as pd

answers_path = "../data/raw/Answers.csv"

answers_path_df = pd.read_csv(
    answers_path,
    encoding="latin1",
    nrows=10000      # chunk
)

answers_path_df.shape

(10000, 6)

In [28]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import html


In [29]:
questions_df.columns


Index(['Id', 'OwnerUserId', 'CreationDate', 'ClosedDate', 'Score', 'Title',
       'Body'],
      dtype='object')

In [30]:
questions_df.head(3)


,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...


In [31]:
answers_path_df.head(3)

,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,92,61.0,2008-08-01T14:45:37Z,90,13,"<p><a href=""http://svnbook.red-bean.com/"">Vers..."
1,124,26.0,2008-08-01T16:09:47Z,80,12,<p>I wound up using this. It is a kind of a ha...
2,199,50.0,2008-08-01T19:36:46Z,180,1,<p>I've read somewhere the human eye can't dis...


In [32]:
questions_df[["Title", "Body"]].iloc[0]


Title    SQLStatement.execute() - multiple queries in o...
Body     <p>I've written a database generation script i...
Name: 0, dtype: object

no of questions /answers/tags in raw dataset

In [33]:
import pandas as pd

questions_path = "../data/raw/Questions.csv"

q_count = 0
for chunk in pd.read_csv(
    questions_path,
    encoding="latin1",
    chunksize=100_000
):
    q_count += len(chunk)

q_count


1264216

In [34]:
answers_path = "../data/raw/Answers.csv"

a_count = 0
for chunk in pd.read_csv(
    answers_path,
    encoding="latin1",
    chunksize=100_000
):
    a_count += len(chunk)

a_count


2014516

In [35]:
tags_path = "../data/raw/Tags.csv"

t_count = 0
for chunk in pd.read_csv(
    tags_path,
    encoding="latin1",
    chunksize=100_000
):
    t_count += len(chunk)

t_count


3750994

In [36]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import html


In [37]:
questions_path = "../data/raw/Questions.csv"

df = pd.read_csv(
    questions_path,
    encoding="latin1",
    nrows=10000
)

In [38]:
df["raw_text"] = (
    df["Title"].fillna("") + " " +
    df["Body"].fillna("")
)


In [39]:
def clean_text_pipeline(text):
    if pd.isna(text):
        return ""
    
    # Decode HTML entities
    text = html.unescape(text)
    
    # Remove HTML/XML tags safely
    text = BeautifulSoup(text, "lxml").get_text()
    
    # Lowercase
    text = text.lower()
    
    # Remove non-letters
    text = re.sub(r"[^a-z\s]", " ", text)
    
    # Normalize spaces
    text = re.sub(r"\s+", " ", text)
    
    return text.strip()



In [40]:
df["clean_text"] = df["raw_text"].apply(clean_text_pipeline)


FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

In [ ]:
sample = df[["raw_text", "clean_text"]].iloc[0]

print("RAW TEXT:\n", sample["raw_text"][:500])
print("\nCLEAN TEXT:\n", sample["clean_text"][:500])


RAW TEXT:
 SQLStatement.execute() - multiple queries in one statement <p>I've written a database generation script in <a href="http://en.wikipedia.org/wiki/SQL">SQL</a> and want to execute it in my <a href="http://en.wikipedia.org/wiki/Adobe_Integrated_Runtime">Adobe AIR</a> application:</p>

<pre><code>Create Table tRole (
      roleID integer Primary Key
      ,roleName varchar(40)
);
Create Table tFile (
    fileID integer Primary Key
    ,fileName varchar(50)
    ,fileDescription varchar(500)
    ,thum

CLEAN TEXT:
 sqlstatement execute multiple queries in one statement i ve written a database generation script in sql and want to execute it in my adobe air application create table trole roleid integer primary key rolename varchar create table tfile fileid integer primary key filename varchar filedescription varchar thumbnailid integer fileformatid integer categoryid integer isfavorite boolean dateadded date globalaccesscount integer lastaccesstime date downloadcomplete boolean isne

In [ ]:
input_path = "../data/raw/Questions.csv"
output_path = "../data/processed/questions_clean.csv"

chunk_size = 50000
first_chunk = True

for chunk in pd.read_csv(
    input_path,
    encoding="latin1",
    chunksize=chunk_size
):
    chunk["raw_text"] = (
        chunk["Title"].fillna("") + " " +
        chunk["Body"].fillna("")
    )
    
    chunk["clean_text"] = chunk["raw_text"].apply(clean_text_pipeline)
    
    processed_chunk = chunk[["Id", "Score", "clean_text"]]
    
    processed_chunk.to_csv(
        output_path,
        mode="w" if first_chunk else "a",
        header=first_chunk,
        index=False
    )
    
    first_chunk = False


In [ ]:
clean_df = pd.read_csv(
    "../data/processed/questions_clean.csv",
    nrows=5
)

clean_df.head()


,Id,Score,clean_text
0,80,26,sqlstatement execute multiple queries in one s...
1,90,144,good branching and merging tutorials for torto...
2,120,21,asp net site maps has anyone got experience cr...
3,180,53,function for creating color wheels this is som...
4,260,49,adding scripting functionality to net applicat...


In [ ]:
clean_df_full = pd.read_csv(
    "../data/processed/questions_clean.csv"
)

clean_df_full.shape


(2278432, 3)

In [ ]:
(clean_df_full["clean_text"].str.len() == 0).sum()


np.int64(0)

In [ ]:
original_count = 0
for chunk in pd.read_csv(
    "../data/raw/Questions.csv",
    encoding="latin1",
    chunksize=100000
):
    original_count += len(chunk)

original_count



1264216

In [ ]:
clean_count = 0
for chunk in pd.read_csv(
    "../data/processed/questions_clean.csv",
    chunksize=100000
):
    clean_count += len(chunk)

clean_count


2278432